This script will reproject files to the TEM 4km grid using bilinear for CMIP6/FWI

CMIP6

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import pathlib
from osgeo import gdal, osr

TEMPLATE = "/explore/nobackup/people/spotter5/clelland_fire_ml/tem_grid.tif"
IN_ROOT  = "/explore/nobackup/people/spotter5/clelland_fire_ml/CMIP6_files"
OUT_ROOT = "/explore/nobackup/people/spotter5/clelland_fire_ml/CMIP6_files_reprojected"

gdal.UseExceptions()
os.environ.setdefault("GDAL_NUM_THREADS", "ALL_CPUS")
gdal.SetConfigOption("GDAL_CACHEMAX", "2048")  # MB

def is_tif(p: pathlib.Path) -> bool:
    return p.is_file() and p.suffix.lower() in (".tif", ".tiff") and not str(p).endswith(".aux.xml")

def read_template_info(path):
    ds = gdal.Open(path, gdal.GA_ReadOnly)
    if ds is None:
        raise RuntimeError(f"Cannot open template: {path}")
    gt = ds.GetGeoTransform()              # (x0, px, rx, y0, ry, py)
    proj_wkt = ds.GetProjectionRef()
    xsize, ysize = ds.RasterXSize, ds.RasterYSize
    x0, px, rx, y0, ry, py = gt
    left   = x0
    top    = y0
    right  = x0 + px * xsize + rx * ysize
    bottom = y0 + ry * xsize + py * ysize
    xmin, xmax = min(left, right), max(left, right)
    ymin, ymax = min(bottom, top), max(bottom, top)
    return {
        "gt": gt,
        "xsize": xsize, "ysize": ysize,
        "bounds": (xmin, ymin, xmax, ymax),
        "proj_wkt": proj_wkt
    }

def main():
    tmpl = read_template_info(TEMPLATE)
    print("Template locked:")
    print("  CRS:", osr.SpatialReference(wkt=tmpl["proj_wkt"]).GetAttrValue("AUTHORITY", 1) or "WKT")
    print("  size (W,H):", tmpl["xsize"], tmpl["ysize"])
    print("  bounds    :", tmpl["bounds"])

    warp_opts = gdal.WarpOptions(
        format="GTiff",
        outputBounds=tmpl["bounds"],
        width=tmpl["xsize"], height=tmpl["ysize"],
        dstSRS=tmpl["proj_wkt"],
        resampleAlg=gdal.GRA_Bilinear,
        multithread=True,
        warpOptions=["NUM_THREADS=ALL_CPUS", "INIT_DEST=NO_DATA"],
        # If your inputs have nodata, set srcNodata accordingly (e.g.,  -9999)
        srcNodata=None,
        dstNodata=float("nan"),
        creationOptions=[
            "COMPRESS=LZW", "PREDICTOR=2", "TILED=YES",
            "BIGTIFF=IF_SAFER", "BLOCKXSIZE=256", "BLOCKYSIZE=256"
        ]
    )

    in_root  = pathlib.Path(IN_ROOT)
    out_root = pathlib.Path(OUT_ROOT)
    out_root.mkdir(parents=True, exist_ok=True)

    tifs = [p for p in in_root.rglob("*") if is_tif(p)]
    if not tifs:
        print("No GeoTIFFs found under", IN_ROOT)
        return

    for src in sorted(tifs):
        rel = src.relative_to(in_root)
        dst = out_root / rel
        dst.parent.mkdir(parents=True, exist_ok=True)
        if dst.exists():
            print(f"[SKIP] {dst}")
            continue

        try:
            gdal.Warp(dst.as_posix(), src.as_posix(), options=warp_opts)
            print(f"[OK] {src} -> {dst}")
        except Exception as e:
            print(f"[ERR] {src} :: {e}")

if __name__ == "__main__":
    main()


Template locked:
  CRS: 6931
  size (W,H): 2242 1934
  bounds    : (-4602000.0, -3485000.0, 4366000.0, 4251000.0)
[OK] /explore/nobackup/people/spotter5/clelland_fire_ml/CMIP6_files/ACCESS-CM2_COG/Historical/ACCESS-CM2_historical_2001_10_all_cog.tif -> /explore/nobackup/people/spotter5/clelland_fire_ml/CMIP6_files_reprojected/ACCESS-CM2_COG/Historical/ACCESS-CM2_historical_2001_10_all_cog.tif
[OK] /explore/nobackup/people/spotter5/clelland_fire_ml/CMIP6_files/ACCESS-CM2_COG/Historical/ACCESS-CM2_historical_2001_11_all_cog.tif -> /explore/nobackup/people/spotter5/clelland_fire_ml/CMIP6_files_reprojected/ACCESS-CM2_COG/Historical/ACCESS-CM2_historical_2001_11_all_cog.tif
[OK] /explore/nobackup/people/spotter5/clelland_fire_ml/CMIP6_files/ACCESS-CM2_COG/Historical/ACCESS-CM2_historical_2001_12_all_cog.tif -> /explore/nobackup/people/spotter5/clelland_fire_ml/CMIP6_files_reprojected/ACCESS-CM2_COG/Historical/ACCESS-CM2_historical_2001_12_all_cog.tif
[OK] /explore/nobackup/people/spotter5/c

FWI

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import pathlib
from osgeo import gdal, osr

TEMPLATE = "/explore/nobackup/people/spotter5/clelland_fire_ml/tem_grid.tif"
IN_ROOT  = "/explore/nobackup/people/spotter5/clelland_fire_ml/FWI_files"
OUT_ROOT = "/explore/nobackup/people/spotter5/clelland_fire_ml/FWI_files_reprojected"

gdal.UseExceptions()
os.environ.setdefault("GDAL_NUM_THREADS", "ALL_CPUS")
gdal.SetConfigOption("GDAL_CACHEMAX", "2048")  # MB

def is_tif(p: pathlib.Path) -> bool:
    return p.is_file() and p.suffix.lower() in (".tif", ".tiff") and not str(p).endswith(".aux.xml")

def read_template_info(path):
    ds = gdal.Open(path, gdal.GA_ReadOnly)
    if ds is None:
        raise RuntimeError(f"Cannot open template: {path}")
    gt = ds.GetGeoTransform()              # (x0, px, rx, y0, ry, py)
    proj_wkt = ds.GetProjectionRef()
    xsize, ysize = ds.RasterXSize, ds.RasterYSize
    x0, px, rx, y0, ry, py = gt
    left   = x0
    top    = y0
    right  = x0 + px * xsize + rx * ysize
    bottom = y0 + ry * xsize + py * ysize
    xmin, xmax = min(left, right), max(left, right)
    ymin, ymax = min(bottom, top), max(bottom, top)
    return {
        "gt": gt,
        "xsize": xsize, "ysize": ysize,
        "bounds": (xmin, ymin, xmax, ymax),
        "proj_wkt": proj_wkt
    }

def main():
    tmpl = read_template_info(TEMPLATE)
    print("Template locked:")
    print("  CRS:", osr.SpatialReference(wkt=tmpl["proj_wkt"]).GetAttrValue("AUTHORITY", 1) or "WKT")
    print("  size (W,H):", tmpl["xsize"], tmpl["ysize"])
    print("  bounds    :", tmpl["bounds"])

    warp_opts = gdal.WarpOptions(
        format="GTiff",
        outputBounds=tmpl["bounds"],
        width=tmpl["xsize"], height=tmpl["ysize"],
        dstSRS=tmpl["proj_wkt"],
        resampleAlg=gdal.GRA_Bilinear,
        multithread=True,
        warpOptions=["NUM_THREADS=ALL_CPUS", "INIT_DEST=NO_DATA"],
        # If your inputs have nodata, set srcNodata accordingly (e.g.,  -9999)
        srcNodata=None,
        dstNodata=float("nan"),
        creationOptions=[
            "COMPRESS=LZW", "PREDICTOR=2", "TILED=YES",
            "BIGTIFF=IF_SAFER", "BLOCKXSIZE=256", "BLOCKYSIZE=256"
        ]
    )

    in_root  = pathlib.Path(IN_ROOT)
    out_root = pathlib.Path(OUT_ROOT)
    out_root.mkdir(parents=True, exist_ok=True)

    tifs = [p for p in in_root.rglob("*") if is_tif(p)]
    if not tifs:
        print("No GeoTIFFs found under", IN_ROOT)
        return

    for src in sorted(tifs):
        rel = src.relative_to(in_root)
        dst = out_root / rel
        dst.parent.mkdir(parents=True, exist_ok=True)
        if dst.exists():
            print(f"[SKIP] {dst}")
            continue

        try:
            gdal.Warp(dst.as_posix(), src.as_posix(), options=warp_opts)
            print(f"[OK] {src} -> {dst}")
        except Exception as e:
            print(f"[ERR] {src} :: {e}")

if __name__ == "__main__":
    main()


In [3]:
# --- Read template grid specs exactly ---
with rio.open(TEMPLATE) as ds:
    tmpl_crs = ds.crs      
    print(tmpl_crs)

EPSG:6931
